In [ ]:
import tfimm

print(tfimm.list_models(pretrained="timm"))

In [ ]:
timm.create_model?

In [1]:
import timm
import os
import numpy as np


root_path = os.getcwd()
model_dir = os.path.join(root_path, 'model', 'vit_timm')
cfg_path = os.path.join(model_dir, 'config.json')
data_dir = os.path.join(root_path, 'dataset', 'Fruit-Images-Dataset', 'Training')
checkpoint_path = os.path.join(model_dir, 'pytorch_model.bin')
model = timm.create_model('vit_base_patch8_224.augreg_in21k', 
                          # pretrained=True, 
                          # pretrained_cfg=cfg_path,
                          checkpoint_path=checkpoint_path)
model.eval() # set the model in evaluation mode
data_config = timm.data.resolve_model_data_config(model)
transforms_timm = timm.data.create_transform(**data_config, is_training=False)

/root/anaconda3/envs/tf214/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
timm.data.create_transform?

Signature:
timm.data.create_transform(
    input_size: Union[int, Tuple[int, int], Tuple[int, int, int]] = 224,
    is_training: bool = False,
    no_aug: bool = False,
    train_crop_mode: Optional[str] = None,
    scale: Optional[Tuple[float, float]] = None,
    ratio: Optional[Tuple[float, float]] = None,
    hflip: float = 0.5,
    vflip: float = 0.0,
    color_jitter: Union[float, Tuple[float, ...]] = 0.4,
    color_jitter_prob: Optional[float] = None,
    grayscale_prob: float = 0.0,
    gaussian_blur_prob: float = 0.0,
    auto_augment: Optional[str] = None,
    interpolation: str = 'bilinear',
    mean: Tuple[float, ...] = (0.485, 0.456, 0.406),
    std: Tuple[float, ...] = (0.229, 0.224, 0.225),
    re_prob: float = 0.0,
    re_mode: str = 'const',
    re_count: int = 1,
    re_num_splits: int = 0,
    crop_pct: Optional[float] = None,
    crop_mode: Optional[str] = None,
    crop_border_pixels: Optional[int] = None,
    tf_preprocessing: bool = False,
    use_prefetcher: bool

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets


# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     # transforms.ToPILImage(),
# ])

device = torch.device("cuda")
model.to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    
# dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataset = datasets.ImageFolder(root=data_dir, transform=transforms_timm)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
for images, labels in dataloader:
    output = model(images.to(device))
    top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=1)
    print(top5_probabilities, top5_class_indices)
    break

tensor([[11.7832],
        [63.2757],
        [32.8497],
        [58.0160],
        [14.2270],
        [17.7258],
        [50.9037],
        [37.7923],
        [33.7876],
        [62.2489],
        [28.0929],
        [18.4387],
        [66.1786],
        [24.0341],
        [85.1479],
        [74.5199],
        [23.6781],
        [12.9566],
        [38.2331],
        [97.1061],
        [86.4187],
        [23.1649],
        [34.7576],
        [33.7956],
        [84.6642],
        [48.0698],
        [80.0134],
        [30.4072],
        [52.8507],
        [53.6827],
        [80.1194],
        [80.1208]], device='cuda:0', grad_fn=<TopkBackward0>) tensor([[17838],
        [13178],
        [21748],
        [19367],
        [13072],
        [13003],
        [20441],
        [19481],
        [13022],
        [13068],
        [13089],
        [13051],
        [19414],
        [13052],
        [20271],
        [21748],
        [19364],
        [20824],
        [13176],
        [19414],
        [

In [3]:
from huggingface_hub import hf_hub_download
import json

repo_id = "huggingface/label-files"
filename = "imagenet-22k-id2label.json"
id2label = json.load(open(filename, "r"))
id2label = {int(k):v for k,v in id2label.items()}

In [19]:
indices = top5_class_indices.cpu().numpy()
for n, i in enumerate(indices):
    print(dataset.classes[labels[n]], i, id2label[i[0]])

Cucumber Ripe [17838] Dutchman's-pipe, pipe_vine, Aristolochia_macrophylla, Aristolochia_durior
Pear 2 [13178] bartlett, bartlett_pear
Chestnut [21748] ovoid
Banana Lady Finger [19367] edible_banana, Musa_paradisiaca_sapientum
Cherry Wax Red [13072] plum
Apple Braeburn [13003] Empire
Pomegranate [20441] mango, mango_tree, Mangifera_indica
Fig [19481] fig, common_fig, common_fig_tree, Ficus_carica
Blueberry [13022] bilberry, whortleberry, European_blueberry
Apricot [13068] apricot
Passion Fruit [13089] passion_fruit
Tangelo [13051] tangerine
Kiwi [19414] Chinese_gooseberry, kiwi, kiwi_vine, Actinidia_chinensis, Actinidia_deliciosa
Tangelo [13052] tangelo, ugli, ugli_fruit
Carambula [20271] carambola, carambola_tree, Averrhoa_carambola
Cucumber Ripe 2 [21748] ovoid
Banana Red [19364] dwarf_banana, Musa_acuminata
Tomato Heart [20824] plum_tomato
Pear 2 [13176] bosc
Kiwi [19414] Chinese_gooseberry, kiwi, kiwi_vine, Actinidia_chinensis, Actinidia_deliciosa
Apple Golden 2 [13001] Golden_Deli